In [4]:
# Import libraries used for the Project
import numpy as np 
import pandas as pd
from scipy import stats
from scipy.stats import skew, kurtosis

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.feature_selection import SelectFromModel

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import tree


import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
X_train = pd.read_csv("X_train.csv")
X_val = pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")
y_test = pd.read_csv("y_test.csv")

In [6]:
# Choose the number of features
top6_features = ['Rain_mm', 'RH_9am', 'RH_3pm', 'Temp_3pm', 'Max_Temp', 'Temp_9am']

PART 7 - SVM

7.1 Parametre Tunning

In [8]:
# We are going to work with the top 6 features and use GridsearchCV to find the best parametres
parametres = {
    #'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    #'gamma': [0.1, 1, 10],
    'degree': [2,3,4],
    #'class_weight': [None, 'balanced']
}

# Now we test the parametres
svc = svm.SVC(kernel="poly")

# Start the Gridsearch
grid_search = GridSearchCV(svc, parametres, cv=5)
grid_search.fit(X_train[top6_features], y_train)

# Now I get to enjoy some waiting time

c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5, estimator=SVC(kernel='poly'),
             param_grid={'degree': [2, 3, 4]})

In [9]:
# Look at the results 
print("Parameters: ", grid_search.best_params_)
print("Score: ", grid_search.best_score_)


# Best result
# Score:  0.8923654077956152
# If I increase to degree 4, we get better result

Parameters:  {'degree': 4}
Score:  0.8787489106302768


In [10]:
# Thinking back at the KNN I also want to test if we can get a better result with only two features and different kernel
linear_svc = svm.SVC(kernel='linear').fit(X_train[top6_features[:2]], y_train)
rbf_svc = svm.SVC(kernel='rbf', gamma=5).fit(X_train[top6_features[:2]], y_train)
poly_svc = svm.SVC(kernel='poly', degree=4).fit(X_train[top6_features[:2]], y_train)

# Scores with 2 Features 
print("Poly:", accuracy_score(y_val, poly_svc.predict(X_val[top6_features[:2]])).round(4))
print("RBF:", accuracy_score(y_val, rbf_svc.predict(X_val[top6_features[:2]])).round(4))
print("Linear:", accuracy_score(y_val, linear_svc.predict(X_val[top6_features[:2]])).round(4))


c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Poly: 0.8715
RBF: 0.81
Linear: 0.7416


In [11]:
# Testing the best result with 3 Features 
poly_svc = svm.SVC(kernel='poly', degree=4).fit(X_train[top6_features[:3]], y_train)
print("Poly:", accuracy_score(y_val, poly_svc.predict(X_val[top6_features[:3]])).round(4))

# Not much a difference

c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Poly: 0.8793


7.2 Final Testing on Validation for Full features or Two Features

In [14]:
# One last test with full features or just two
poly_svc_full = svm.SVC(kernel='poly', degree=4, C=5).fit(X_train[top6_features], y_train)
poly_svc_2 = svm.SVC(kernel='poly', degree=4, C=5).fit(X_train[top6_features[:2]], y_train)

# Scores
print("Poly Full:", accuracy_score(y_val, poly_svc_full.predict(X_val[top6_features])).round(4))
print("Poly Two:", accuracy_score(y_val, poly_svc_2.predict(X_val[top6_features[:2]])).round(4))

c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jmanu\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Poly Full: 0.8929
Poly Two: 0.8938


7.3 Testing With Final Model

In [ ]:
# Now that we finally have the parametres let's test
svc = svm.SVC(kernel='poly', degree=4)
svc.fit(X_train[top6_features], y_train)
y_pred = svc.predict(X_test[top6_features])

# Score
print(f"Testing Accuracy: {accuracy_score(y_pred, y_test).round(4) * 100}%")

7.4 Visualisation of the Result

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix, display_labels=['No Rain', 'Rain'])
disp.plot(cmap='Blues', xticks_rotation='horizontal')
disp.ax_.set_xlabel('Predicted label', fontsize=12)
disp.ax_.set_ylabel('True label', fontsize=12)
plt.grid(False)
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))